# TCC - Genetic Algorithm to control Waves in Games

### Authors:
 - Daniel Hotta
 - Rafael Gonçalves Pereira Silva
 - Ricardo Akira Tanaka

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as mpgs
from matplotlib.backends.backend_pdf import PdfPages


## Simple description

This notebook uses the results of the experiments we made with our Genetic Algorithm to conclude the overall perfomance against a Uniform Enemy Generator and "Naive Strategies" (Like All Enemies with the same color or One of Each color).
For uniform enemies, 10 waves were enough data, otherwise (random and AI) 30 waves were collected.

- The libs you can find in two repositories: [SpaceShip](https://github.com/RGPRafael/godot) and [Tower Defense](https://github.com/raktanaka/tccTD)

- The full dataset can be found [here](https://github.com/raktanaka/tcc-results).




In [2]:
def GetFilelist():
    td_files = []
    ss_files = []

    for subdir, dirs, files in os.walk('.'):
        for file in files:
            if subdir[2:] == 'tccTD' and filepath.endswith('.txt'):
                filepath = subdir + os.sep + file
                td_files.append(filepath)
            elif subdir[2:] == 'tccSS' and filepath.endswith('.txt'):
                filepath = subdir + os.sep + file
                ss_files.append(filepath)
            else:
                pass
    return(td_files, ss_files)

Reads txt data, trimmings results for 300 lines in case of 10 waves, or 900 lines for 30 waves.

In [3]:
def GetFileData(filepath):
  df = pd.read_csv(filepath, header=None, delimiter=';')

  if df.shape[0] < 899:
    return(df.iloc[:300])
  else:
    return(df.iloc[:900])
    

In [4]:
td_allred_ai_file = os.path.join('tccTD', 'AllRed - AI.txt')
td_allred_rd_file = os.path.join('tccTD', 'AllRed - Random.txt')

#td_allgreen_ai_file = os.path.join('tccTD', 'AllGreen - AI.txt')
#td_allgreen_rd_file = os.path.join('tccTD', 'AllGreen - Random.txt')

td_greenred_ai_file = os.path.join('tccTD', 'GreenRed - AI.txt')
td_greenred_rd_file = os.path.join('tccTD', 'GreenRed - Random.txt')

td_redgreen_ai_file = os.path.join('tccTD', 'RedGreen - AI.txt')
td_redgreen_rd_file = os.path.join('tccTD', 'RedGreen - Random.txt')

In [5]:
ss_yellowmove_ai_file = os.path.join('tccSS', 'IA_MOVING_ SHOOTING _YELLOW_SHOOT - AI.txt')
ss_yellowmove_rd_file = os.path.join('tccSS', 'IA_MOVING_ SHOOTING _YELLOW_SHOOT - Random.txt')

ss_yellowstill_ai_file = os.path.join('tccSS', 'IA_STILL_YELLOW_SHOOT - AI.txt')
ss_yellowstill_rd_file = os.path.join('tccSS', 'IA_STILL_YELLOW_SHOOT - Random.txt')

ss_redmove_ai_file = os.path.join('tccSS', 'IA_MOVING_SHOOTING_ RED_SHOOT - AI.txt')
ss_redmove_rd_file = os.path.join('tccSS', 'IA_MOVING_SHOOTING_ RED_SHOOT - Random.txt')

#ss_redstill_ai_file = os.path.join('tccSS', 'IA_STILL_RED_SHOOT - AI.txt')
#ss_redstill_rd_file = os.path.join('tccSS', 'IA_STILL_RED_SHOOT - Random.txt')

Tower Defense data "header"
0: wave number
1-12: element in wave
13: damage done in wave

In [6]:
td_allred_ai = GetFileData(td_allred_ai_file)
td_allred_rd = GetFileData(td_allred_rd_file)

#td_allgreen_ai = GetFileData(td_allgreen_ai_file)
#td_allgreen_rd = GetFileData(td_allgreen_rd_file)

td_greenred_ai = GetFileData(td_greenred_ai_file)
td_greenred_rd = GetFileData(td_greenred_rd_file)

td_redgreen_ai = GetFileData(td_redgreen_ai_file)
td_redgreen_rd = GetFileData(td_redgreen_rd_file)

FileNotFoundError: [Errno 2] No such file or directory: 'tccTD/AllRed - AI.txt'

In [ ]:
ss_yellowmove_ai = GetFileData(ss_yellowmove_ai_file)
ss_yellowmove_rd = GetFileData(ss_yellowmove_rd_file)

ss_yellowstill_ai = GetFileData(ss_yellowstill_ai_file)
ss_yellowstill_rd = GetFileData(ss_yellowstill_rd_file)

ss_redmove_ai = GetFileData(ss_redmove_ai_file)
ss_redmove_rd = GetFileData(ss_redmove_rd_file)

#ss_redstill_ai = GetFileData(ss_redstill_ai_file)
#ss_redstill_rd = GetFileData(ss_redstill_rd_file)

In [ ]:
def GenerateHeader(df):
    header = ['wave number']

    if df.shape[1] < 14:
        n_enemies = 6
    else:
        n_enemies = 12

    for each in range(n_enemies):
        header.append(str('enemy ' + str(each + 1)))

    header.append('total damage')
    return(header)

In [ ]:
td_allred_ai.columns = GenerateHeader(td_allred_ai)
td_allred_rd.columns = GenerateHeader(td_allred_rd)

#td_allgreen_ai.columns(GenerateHeader(td_allgreen_ai)
#td_allgreen_rd.columns(GenerateHeader(td_allgreen_rd)

td_greenred_ai.columns = GenerateHeader(td_greenred_ai)
td_greenred_rd.columns = GenerateHeader(td_greenred_rd)

td_redgreen_ai.columns = GenerateHeader(td_redgreen_ai)
td_redgreen_rd.columns = GenerateHeader(td_redgreen_rd)

In [ ]:
ss_yellowmove_ai.columns = GenerateHeader(ss_yellowmove_ai)
ss_yellowmove_rd.columns = GenerateHeader(ss_yellowmove_rd)

ss_yellowstill_ai.columns = GenerateHeader(ss_yellowstill_ai)
ss_yellowstill_rd.columns = GenerateHeader(ss_yellowstill_rd)

ss_redmove_ai.columns = GenerateHeader(ss_redmove_ai)
ss_redmove_rd.columns = GenerateHeader(ss_redmove_rd)

#ss_redstill_ai.columns = GenerateHeader(ss_redstill_ai)
#ss_redstill_rd.columns = GenerateHeader(ss_redstill_rd)

td_allred_ai
td_allred_rd

#td_allgreen_ai
#td_allgreen_rd

td_greenred_ai
td_greenred_rd

td_redgreen_ai
td_redgreen_rd

In [ ]:
def CalcStats(series):
    res = []
    res.append(series.mean())
    res.append(series.std())

    return(res)

Calculates the mode for each wave considering the 30 experiments

In [ ]:
def CalcMode(df):
    
    res = []

    for i in range(1, 31):
        # Filtering per wave number (30 repetitions of wave i)
        tmp_df = df[df['wave number'] == i]
        # Calculating the mode
        l = tmp_df.iloc[:, 0:-1].mode().iloc[[0]]
        # Averaging the damage
        l['average damage'] = f"{tmp_df['total damage'].mean():.2f}"
        res.append(l)

    new_df = pd.concat(res, ignore_index=True)
    return(new_df)

In [ ]:
#td_allgreen_ai_mode = CalcMode(td_allgreen_ai)
td_allred_ai_mode = CalcMode(td_allred_ai)
td_greenred_ai_mode = CalcMode(td_greenred_ai)
td_redgreen_ai_mode = CalcMode(td_redgreen_ai)

In [ ]:
ss_yellowmove_ai_mode = CalcMode(ss_yellowmove_ai)
ss_yellowstill_ai_mode = CalcMode(ss_yellowstill_ai)
ss_redmove_ai_mode = CalcMode(ss_redmove_ai)
#ss_redstill_ai_mode = CalcMode(ss_redstill_ai)

Auxiliar functions to clean characters and split the enemy and direction from the dataframe

In [ ]:
def RemoveChar(str):

    ch_rem = '[] '
    for i in ch_rem:
        str = str.replace(i, '')
    
    str = str.split(',')
    return(str[0], str[1])

def ListEnemies(df_row):

    enemy_list = []
    direction_list = []
    
    for each in df_row:
        enemy, direction = RemoveChar(each)
        enemy_list.append(enemy)
        direction_list.append(direction)

    return(enemy_list, direction_list)

def GetImage(enemy, direction):

    #img_td_blue = mpimg.imread(os.path.join('sprites', 'tank_blue.png'))
    #img_td_green = mpimg.imread(os.path.join('sprites', 'tank_green.png'))
    #img_td_orange = mpimg.imread(os.path.join('sprites', 'tank_orange.png'))
    #img_td_purple = mpimg.imread(os.path.join('sprites', 'tank_purple.png'))
    #img_td_red = mpimg.imread(os.path.join('sprites', 'tank_red.png'))
    #img_td_yellow = mpimg.imread(os.path.join('sprites', 'tank_yellow.png'))

    #img_ss_inimigos = mpimg.imread(os.path.join('sprites', 'inimigos.png'))
    #img_ss_inimigo1 = mpimg.imread(os.path.join('sprites', 'inimigo1.png'))
    #img_ss_inimigo2 = mpimg.imread(os.path.join('sprites', 'inimigo2.png'))
    #img_ss_inimigo3 = mpimg.imread(os.path.join('sprites', 'inimigo3.png'))
    #img_ss_inimigo4 = mpimg.imread(os.path.join('sprites', 'inimigo4.png'))
    #img_ss_inimigo5 = mpimg.imread(os.path.join('sprites', 'inimigo5.png'))

    if enemy == 'EnemyRed':
        img = mpimg.imread(os.path.join('sprites', 'tank_red.png'))
    elif enemy == 'EnemyGreen':
        img = mpimg.imread(os.path.join('sprites', 'tank_green.png'))
    elif enemy == 'EnemyBlue':
        img = mpimg.imread(os.path.join('sprites', 'tank_blue.png'))
    elif enemy == 'EnemyYellow':
        img = mpimg.imread(os.path.join('sprites', 'tank_yellow.png'))
    elif enemy == 'EnemyPurple':
        img = mpimg.imread(os.path.join('sprites', 'tank_purple.png'))
    elif enemy == 'EnemyOrange':
        img = mpimg.imread(os.path.join('sprites', 'tank_orange.png'))

    elif enemy == 'inimigos':
        img = mpimg.imread(os.path.join('sprites', 'inimigos.png'))
    elif enemy == 'inimigo1':
        img = mpimg.imread(os.path.join('sprites', 'inimigo1.png'))
    elif enemy == 'inimigo2':
        img = mpimg.imread(os.path.join('sprites', 'inimigo2.png'))
    elif enemy == 'inimigo3':
        img = mpimg.imread(os.path.join('sprites', 'inimigo3.png'))
    elif enemy == 'inimigo4':
        img = mpimg.imread(os.path.join('sprites', 'inimigo4.png'))
    elif enemy == 'inimigo5':
        img = mpimg.imread(os.path.join('sprites', 'inimigo5.png'))

    return(img, '0')


Receives a row of the dataframe to generate the wave image

In [ ]:
def ImageWave(df, filename):
    n_lin, n_col = (df.shape)
    n_col -= 1

    # Checks for 12 (TD) or 6 (SS) enemies per wave
    if n_col < 12:
        # Space Shooter
        #img_dpi = 200
        bg_color = 'xkcd:dark grey'
        font_color = 'white'
    else:
        # Tower Defense
        #img_dpi = 200
        bg_color = 'xkcd:light grey'
        font_color = 'black'

    #heights = [1] * n_lin
    fig = plt.figure(dpi=300, tight_layout=True)
    fig.patch.set_facecolor(bg_color)
    fig.set_size_inches(4, 24, forward=True)
    ax = np.zeros(n_col, dtype=object)
    gs = fig.add_gridspec(n_lin, n_col)
    # Loops each i wave mode, generating the images
    for i in range(n_lin):
        df_row = df.iloc[i, 1:-1]
        enemy_list, direction_list = ListEnemies(df_row)
        for j in range(n_col):
            ax[j] = fig.add_subplot(gs[i, j])
            plt.axis('off')
            if j == 0:
                ax[j].text(0.45, 0.45, str(i + 1), fontsize='xx-small', fontweight='bold', color=font_color)
            elif j == (n_col - 1):
                ax[j].text(0.45, 0.45, str(df.iloc[i, -1]), fontsize='xx-small', fontweight='bold', color=font_color)
            else:
            #ax[j].set_facecolor('xkcd:grey')
                #plt.axis('off')
                img, dir = GetImage(enemy_list[j], direction_list[j])
                ax[j].imshow(img)
    plt.close()
    filename += '.png'
    fig.savefig(filename)


In [ ]:
#ImageWave(td_allgreen_ai_mode, 'td_allgreen_ai')
ImageWave(td_allred_ai_mode, 'td_allred_ai')
ImageWave(td_greenred_ai_mode, 'td_greenred_ai')
ImageWave(td_redgreen_ai_mode, 'td_redgreen_ai')

In [ ]:
ImageWave(ss_yellowmove_ai_mode, 'ss_yellowmove_ai')
ImageWave(ss_yellowstill_ai_mode, 'ss_yellowstill_ai')
ImageWave(ss_redmove_ai_mode, 'ss_redmove_ai')
#ImageWave(ss_redstill_ai_mode, 'ss_redstill_ai')